In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
from google.colab import files

In [3]:
uploaded= files.upload()

Saving dataset.zip to dataset.zip


In [5]:
import zipfile
import io
zf = zipfile.ZipFile(io.BytesIO(uploaded['dataset.zip']), "r")
zf.extractall()

In [6]:
os.listdir('Road Data')

['Test', 'Train']

In [7]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [8]:
val_datagen = ImageDataGenerator(rescale =1.0/255.)

In [9]:
training_set1 = train_datagen.flow_from_directory('Road Data/Train',
                                                 target_size = (100, 100),
                                                 batch_size = 10,
                                                 class_mode = 'binary')

Found 24 images belonging to 2 classes.


In [10]:
training_set2 = train_datagen.flow_from_directory('Traffic Data/Train',
                                                 target_size = (100, 100),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

Found 21 images belonging to 3 classes.


In [11]:
val_set1 = val_datagen.flow_from_directory('Road Data/Test',
                                            target_size = (100, 100),
                                            batch_size = 8,
                                            class_mode = 'binary')

Found 14 images belonging to 2 classes.


In [12]:
val_set2 = train_datagen.flow_from_directory('Traffic Data/Test',
                                                 target_size = (100, 100),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

Found 6 images belonging to 3 classes.


In [13]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-01-03 09:39:43--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 74.125.203.128, 74.125.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   134MB/s    in 0.6s    

2021-01-03 09:39:44 (134 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [14]:
local_weight_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [15]:
pretrained_model = InceptionV3(input_shape= (100,100,3),include_top = False,weights = None)

pretrained_model.load_weights(local_weight_file)

for layer in pretrained_model.layers:
    layer.trainable = False

#pretrained_model.summary()

last_layer = pretrained_model.get_layer('mixed8')
last_output = last_layer.output



In [16]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('val_acc')>=0.70):
            print("\nReached 70% accuracy so cancelling training!")
            self.model.stop_training = True

In [17]:
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation = 'relu')(x)
x = layers.Dropout(.2)(x)
x = layers.Dense(1,activation ='sigmoid')(x)
model1 = Model(pretrained_model.input,x)

model1.compile(optimizer=RMSprop(lr=0.01), loss = 'binary_crossentropy', metrics = 'acc')

In [18]:
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation = 'relu')(x)
x = layers.Dropout(.2)(x)
x = layers.Dense(3,activation ='softmax')(x)
model2 = Model(pretrained_model.input,x)

model2.compile(
    optimizer='adam', 
    loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics=['acc'])

In [19]:
my_callback_object = myCallback()
history1 = model1.fit_generator(
    training_set1,validation_data =val_set1,epochs=100, verbose =1,callbacks=[my_callback_object])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
3/3 [==============================] - 6s 1s/step - loss: 22.8082 - acc: 0.4571 - val_loss: 20.7638 - val_acc: 0.5000
Epoch 2/100
3/3 [==============================] - 1s 448ms/step - loss: 22.4224 - acc: 0.6333 - val_loss: 3.7513 - val_acc: 0.7143

Reached 70% accuracy so cancelling training!


In [20]:
history2 = model2.fit_generator(
    training_set2,validation_data = val_set2,
    epochs=100, verbose =1,callbacks=[my_callback_object])

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


3/3 [==============================] - 6s 1s/step - loss: 2.2048 - acc: 0.2860 - val_loss: 2.5296 - val_acc: 0.3333
Epoch 2/100
3/3 [==============================] - 1s 471ms/step - loss: 1.6181 - acc: 0.6234 - val_loss: 1.8096 - val_acc: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 357ms/step - loss: 1.4776 - acc: 0.5234 - val_loss: 2.9482 - val_acc: 0.3333
Epoch 4/100
3/3 [==============================] - 1s 485ms/step - loss: 1.5141 - acc: 0.6721 - val_loss: 2.0570 - val_acc: 0.5000
Epoch 5/100
3/3 [==============================] - 1s 367ms/step - loss: 1.0645 - acc: 0.6185 - val_loss: 1.5614 - val_acc: 0.5000
Epoch 6/100
3/3 [==============================] - 1s 286ms/step - loss: 0.7602 - acc: 0.6946 - val_loss: 1.3619 - val_acc: 0.5000
Epoch 7/100
3/3 [==============================] - 1s 461ms/step - loss: 1.1198 - acc: 0.6082 - val_loss: 1.8112 - val_acc: 0.5000
Epoch 8/100
3/3 [==============================] - 1s 280ms/step - loss: 0.5406 - acc: 0.8423 

In [21]:
!mkdir -p saved_model
model1.save('saved_model/model1')

INFO:tensorflow:Assets written to: saved_model/model1/assets


In [22]:
model2.save('saved_model/modle2')

INFO:tensorflow:Assets written to: saved_model/modle2/assets


In [ ]:
from keras.models import model_from_json
model1_json = model1.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model1_json)
model.save_weights("model1.h5")

In [ ]:
from keras.models import model_from_json
model2_json = model2.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model2_json)
model.save_weights("model2.h5")

In [ ]:
model1 = ['good','bad']
model2 =['high','mod','low']
curr_speed = [i for i in range(0,120)]
opp_distance = [i for i in range(0,100)]
vis_ = [0,1]
time =['mrn','aft','evn','nig']

In [ ]:
from itertools import product

In [ ]:
'''
df =[[i,j,k,l,m,n] for i in model1 for j in model2 for k in curr_speed for l in opp_distance for m in vis_ for n in time]
data_frame = pd.DataFrame(data =df, columns = ['Road_cond','Traffic_cond','curr_speed','opp_dist','vis','time'])

check = data_frame.copy()

list_ = []

for i in range(len(check)):
    if check.loc[i]['vis'] == 0:
        list_.append('E')
    elif check.loc[i]['Traffic_cond'] == 'high' and check.loc[i]['Road_cond'] == 'good' and check.loc[i]['opp_dist']>20:
        list_.append('D')
    elif check.loc[i]['Road_cond'] == 'high' and check.loc[i]['Traffic_cond'] == 'good' and check.loc[i]['opp_dist']>50:
        list_.append('C')
    elif check.loc[i]['opp_dist'] >20 and check.loc[i]['Road_cond'] == 'good' and check.loc[i]['Traffic_cond'] == 'mod':
        list_.append('C')
    elif check.loc[i]['Traffic_cond'] == 'high' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='good' and check.loc[i]['opp_dist']>60:
        list_.append('B')
    elif check.loc[i]['Traffic_cond'] == 'mod' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='good' and check.loc[i]['opp_dist']>50:
        list_.append('B')
    elif check.loc[i]['Traffic_cond'] == 'mod' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='good' and check.loc[i]['opp_dist']>30:
        list_.append('C')
    elif check.loc[i]['Traffic_cond'] == 'high' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='good' and check.loc[i]['opp_dist']<30:
        list_.append('C')
    elif check.loc[i]['Traffic_cond'] == 'high' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='good' and check.loc[i]['opp_dist']<20:
        list_.append('D')
    elif check.loc[i]['Traffic_cond'] == 'low' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='good' and check.loc[i]['opp_dist']>50:
        list_.append('A')
    elif check.loc[i]['Traffic_cond'] == 'mod' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='good' and check.loc[i]['opp_dist']>50:
        list_.append('A')
    elif check.loc[i]['Traffic_cond'] == 'low' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='bad' and check.loc[i]['opp_dist']>20:
        list_.append('C')
    elif check.loc[i]['Traffic_cond'] == 'mod' and check.loc[i]['vis'] == 1 and check.loc[i]['Road_cond']=='bad' and check.loc[i]['opp_dist']>20:
        list_.append('C')
    else:
        list_.append('E')


check['Recommended'] = list_
check.to_csv('ML_data.csv')

files.download('ML_data.csv')
'''

In [ ]:
def check_road(i):
    if i == 'good':
        return(1)
    return 0
def check_traffic(i):
    if i =='high':
        return(2)
    elif i =='mod':
        return(1)
    return 0
check['Road_cond'] =check['Road_cond'].apply(lambda x: check_road(x))
check['Traffic_cond'] = check['Traffic_cond'].apply(lambda x: check_traffic(x))


check_dummy = pd.get_dummies(check.iloc[:,:-1])
X =check_dummy.values
y = check.iloc[:,-1].values

from sklearn.preprocessing import LabelEncoder as Le
encoder = Le()
y_enc = encoder.fit_transform(y)

from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test = tts(X,y_enc,test_size = 0.3)

from sklearn.ensemble import RandomForestClassifier as RFC
cls = RFC()
cls.fit(X_train,y_train)

pred = cls.predict(X_test)

from sklearn.metrics import accuracy_score as acc
print(acc(y_test,pred))

import pickle
filename = 'model.sav'
pickle.dump(cls,open(filename,'wb'))

1.0
